# Creating an RL Based ABR Streaming Algorithm 

In [1]:
import sys
import os
import argparse
import random
import math
import io
import contextlib

import sabre
import numpy as np
import pandas as pd

## Load the Dataset

In [2]:
def get_files(d): 
    return [os.path.join(d, f) for f in os.listdir(d)]
    
def get_movie_manifest(movie_filename):
    manifest = sabre.load_json(movie_filename)                                                                                                                                                        
    manifest = sabre.ManifestInfo(segment_time = manifest['segment_duration_ms'], 
                    bitrates    = manifest['bitrates_kbps'],                                         
                    utilities    = [math.log(b) - math.log(manifest['bitrates_kbps'][0]) for b in manifest['bitrates_kbps']], 
                    segments     = manifest['segment_sizes_bits'])
    return manifest
    
network_hd_dir = '../data/hd_fs'
network_sd_dir = '../data/sd_fs'
network_3G_dir = '../data/3Glogs'
network_4G_dir = '../data/4Glogs'

network_traces = {
    'hd_traces':get_files(network_hd_dir),
    'sd_traces':get_files(network_sd_dir),
    '3G_traces':get_files(network_3G_dir),
    '4G_traces':get_files(network_4G_dir),
}

movies = ['../data/bbb.json',] #  '../data/bbb4k.json']

# for now seed the random in order to save the trained model one day
# and then re-run the code to test another day
random.seed(69420) 

n_views_per_trace = 1
n_chunks = sum(len(get_movie_manifest(m).segments) for m in movies)
print('Summary of Network Data:')
test_train_split = 0.75
for k, v in network_traces.items():
    np.random.shuffle(v)
    idx = int(np.floor(test_train_split * len(v)))
    n_train = n_views_per_trace * n_chunks * idx
    n_test  = n_chunks * (len(v) - idx)
    network_traces[k] = {
        'train': v[:idx],
        'train_steps': n_train,
        'test':  v[idx:],
        'test_steps': n_test
    }
    print(f'  {k}: ')
    print(f'    Training {idx} samples, {n_train} steps to train')
    print(f'    Testing  {len(v) - idx} samples, {n_test}  steps to test')
    print('')

DATA = network_traces['hd_traces']

Summary of Network Data:
  hd_traces: 
    Training 750 samples, 149250 steps to train
    Testing  250 samples, 49750  steps to test

  sd_traces: 
    Training 750 samples, 149250 steps to train
    Testing  250 samples, 49750  steps to test

  3G_traces: 
    Training 64 samples, 12736 steps to train
    Testing  22 samples, 4378  steps to test

  4G_traces: 
    Training 30 samples, 5970 steps to train
    Testing  10 samples, 1990  steps to test



## Train and save several models

In [3]:
from environment import ABR_Env

qoe_alpha, qoe_beta, qoe_delta = (1, 1, 1)

env = ABR_Env(
    DATA['train'],
    movies,
    r_multipliers=[qoe_alpha, qoe_beta, qoe_delta],
)
env_train = env.get_sb_env()

In [4]:
from stable_baselines3 import A2C

model_a2c = A2C('MultiInputPolicy', env_train, verbose=1, device='cpu')
model_a2c.learn(total_timesteps=DATA['train_steps'])
model_a2c.save('../models/a2c.model')

Using cpu device
------------------------------------
| time/                 |          |
|    fps                | 175      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.4     |
|    explained_variance | 0.0566   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 5.07     |
|    value_loss         | 114      |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 174      |
|    iterations         | 200      |
|    time_elapsed       | 5        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.37    |
|    explained_variance | 0.357    |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | 1.53     |
|    value_loss      

KeyboardInterrupt: 

## Test the ABR Algorithms and Gather Results

The argparse arguments were copied and pased from the modified `sabre.py` file. Note, minor changes were made where any variable that was used had to be changed to sabre.var_name. 

In [5]:
parser = argparse.ArgumentParser(description = 'Simulate an ABR session.',
                                 formatter_class = argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('-n', '--network', metavar = 'NETWORK', default = 'network.json',
                    help = 'Specify the .json file describing the network trace.')
parser.add_argument('-nm', '--network-multiplier', metavar = 'MULTIPLIER',
                    type = float, default = 1,
                    help = 'Multiply throughput by MULTIPLIER.')
parser.add_argument('-m', '--movie', metavar = 'MOVIE', default = 'movie.json',
                    help = 'Specify the .json file describing the movie chunks.')
parser.add_argument('-ml', '--movie-length', metavar = 'LEN', type = float, default = None,
                    help = 'Specify the movie length in seconds (use MOVIE length if None).')
parser.add_argument('-a', '--abr', metavar = 'ABR',
                    default = sabre.abr_default,
                    help = 'Choose ABR algorithm from predefined list (%s), or specify .py module to import.' % ', '.join(sabre.abr_list.keys()))
parser.add_argument('-ab', '--abr-basic', action = 'store_true',
                    help = 'Set ABR to BASIC (ABR strategy dependant).')
parser.add_argument('-ao', '--abr-osc', action = 'store_true',
                    help = 'Set ABR to minimize oscillations.')
parser.add_argument('-gp', '--gamma-p', metavar = 'GAMMAP', type = float, default = 5,
                    help = 'Specify the (gamma p) product in seconds.')
parser.add_argument('-noibr', '--no-insufficient-buffer-rule', action = 'store_true',
                    help = 'Disable Insufficient Buffer Rule.')
parser.add_argument('-ma', '--moving-average', metavar = 'AVERAGE',
                    choices = sabre.average_list.keys(), default = sabre.average_default,
                    help = 'Specify the moving average strategy (%s).' %
                    ', '.join(sabre.average_list.keys()))
parser.add_argument('-ws', '--window-size', metavar = 'WINDOW_SIZE',
                    nargs = '+', type = int, default = [3],
                    help = 'Specify sliding window size.')
parser.add_argument('-hl', '--half-life', metavar = 'HALF_LIFE',
                    nargs = '+', type = float, default = [3, 8],
                    help = 'Specify EWMA half life.')
parser.add_argument('-s', '--seek', nargs = 2, metavar = ('WHEN', 'SEEK'),
                    type = float, default = None,
                    help = 'Specify when to seek in seconds and where to seek in seconds.')
choices = ['none', 'left', 'right']
parser.add_argument('-r', '--replace', metavar = 'REPLACEMENT',
                    #choices = choices,
                    default  =  'none',
                    help = 'Set replacement strategy from predefined list (%s), or specify .py module to import.' % ', '.join(choices))
parser.add_argument('-b', '--max-buffer', metavar = 'MAXBUFFER', type = float, default = 25,
                    help = 'Specify the maximum buffer size in seconds.')
parser.add_argument('-noa', '--no-abandon', action = 'store_true',
                    help = 'Disable abandonment.')
parser.add_argument('-rmp', '--rampup-threshold', metavar = 'THRESHOLD',
                    type = int, default = None,
                    help = 'Specify at what quality index we are ramped up (None matches network).')
parser.add_argument('-v', '--verbose', action = 'store_true',
                    help = 'Run in verbose mode.')

_StoreTrueAction(option_strings=['-v', '--verbose'], dest='verbose', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='Run in verbose mode.', metavar=None)

### Gathering the Data
Now that the args have been defined to run sabre, the `test_algorithm` function was written to wrap the call. The results are then saved in a data frame for the given traces and movies to test. 

In [6]:
def parse_sabre(text):
    data = {}
    for line in text.splitlines():
        line = line.split(':')
        assert len(line) == 2
        metric, value = line[0], float(line[1].strip())
        data[metric] = value
    return data

def test_algorithm(movies, traces, abr_algo, print_progress=True) -> pd.DataFrame:
    if print_progress:
        print(f'getting results for: {abr_algo}')

    data = []
    for m in movies:
        for t in traces:
            data_row = {'movie':m, 'network_trace':t, 'algorithm':abr_algo}
            # parse the args for sabre to run. 
            # This would normally be done via cli, 
            # but it is convienient here to do in the notebook
            args = parser.parse_args([
                '--abr', abr_algo,
                '--movie', m,
                '--network', t
            ])
            # capture the results that sabre prints to stdout
            with io.StringIO() as buf, contextlib.redirect_stdout(buf):
                sabre.main(args)
                stdout = buf.getvalue() 
            # parse the stdout text to a more consumable dict format
            results   = parse_sabre(stdout)
            data_row.update(results)
            data.append(data_row)
    return pd.DataFrame(data)


bola_results    = test_algorithm(movies, DATA['test'], 'bola')
dynamic_results = test_algorithm(movies, DATA['test'], 'dynamic')

results_df = pd.concat([bola_results, dynamic_results])

print('data frame shape: ', results_df.shape)
print('columns: ', results_df.columns)

results_df.to_csv('../results/results_df.csv', index=False)

getting results for: bola
getting results for: dynamic
data frame shape:  (500, 27)
columns:  Index(['movie', 'network_trace', 'algorithm', 'buffer size',
       'total played utility', 'time average played utility',
       'total played bitrate', 'time average played bitrate',
       'total play time', 'total play time chunks', 'total rebuffer',
       'rebuffer ratio', 'time average rebuffer', 'total rebuffer events',
       'time average rebuffer events', 'total bitrate change',
       'time average bitrate change', 'total log bitrate change',
       'time average log bitrate change', 'time average score',
       'over estimate count', 'over estimate', 'leq estimate count',
       'leq estimate', 'estimate', 'rampup time', 'total reaction time'],
      dtype='object')
